In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/project/"


Evaluation of MACS and HMMRATAC

In [ ]:
predictions = open(path + 'Macs_29_to_30.bed','r').read()
#change to HMM_out_29_to_30.bed for evaluating HMMRATAC
lines = predictions.split('\n')
predictions = {}
# print(len(lines))
for i in lines:
  j = i.split('\t')
  if(len(j) == 1):
    continue
  if (not j[0] in predictions):
    predictions[j[0]] = []
  
  # print(j)
  if(len(j) < 3):
    print(j)
  predictions[j[0]].append((int(j[1]),int(j[2])))

In [ ]:
TN = 0
FP = 0
real_negatives = open(path + 'real_negative.bed','r').read()
lines = real_negatives.split('\n')
print(len(lines))
RN = {}
for i in lines:
  j = i.split('\t')
  if(len(j) == 1):
    continue
  if (not j[0] in RN ):
    RN[j[0]] = []
  # print(j)
  if(len(j) < 3):
    print(j)
  RN[j[0]].append((int(j[1]),int(j[2])))

real_positives = open(path + 'real_positive.bed','r').read()
lines = real_positives.split('\n')
print(len(lines))
RP = {}
for i in lines:
  j = i.split('\t')
  if(len(j) == 1):
    continue
  # print(j)
  if (not  j[0] in RP):
    RP[j[0]] = []
  if(len(j) < 3):
    print(j)
  RP[j[0]].append((int(j[1]),int(j[2])))




25
22


In [ ]:
max_positions = 0
for i in predictions:
  max_positions = max(max(max(predictions[i])),max_positions)
for i in RN:
  max_positions = max(max(max(RN[i])),max_positions)
for i in RP:
  max_positions = max(max(max(RP[i])),max_positions)


In [ ]:
def total_positions(peak):
  return peak[1] - peak[0] + 1  


In [ ]:
chromosomes = predictions.keys()

In [ ]:
data = {}
count = 0
for chr in chromosomes:
  labels = [-1]*(max_positions+1)
  if (not chr in RN):
    continue
  if (not chr in RP):
    continue
  for i in RN[chr]:
    labels[i[0]:i[1] + 1] = [0]*total_positions(i)

  for i in RP[chr]:
    for j in range(i[1] - i[0] + 1):
      if (labels[j + i[0]] == 0):
        labels[j+i[0]] = -1
        count += 1
      else:
        labels[j+i[0]] = 1
  preds = [0]*(max_positions + 1)
  for i in predictions[chr]:
    preds[i[0]:i[1] + 1] = [1]*total_positions(i)
  labels = np.array(labels)
  preds = np.array(preds)

  preds = preds[labels != -1]
  labels = labels[labels != -1]
  
  TP = (preds[labels == 1] == 1).sum()
  FN = (preds[labels == 1] == 0).sum()
  TN = (preds[labels == 0] == 0).sum()
  FP = (preds[labels == 0] == 1).sum()
  data[chr] = {'TP':TP,'TN':TN,'FP':FP,'FN':FN}

In [ ]:
total_tp = 0
total_fp = 0 
total_fn = 0
total_tn = 0
for chr in chromosomes:
  if (not chr in data):
    continue
  total_tp += data[chr]['TP']
  total_fp += data[chr]['FP']
  total_fn += data[chr]['FN']
  total_tn += data[chr]['TN']

precision = total_tp / (total_tp + total_fp)
recall = total_tp / (total_tp + total_fn)
print('precision = ',precision)
print('recall = ',recall)

precision =  1.0
recall =  0.00030615815255423374


In [ ]:
print(total_tp)
print(total_tn)
print(total_fp)
print(total_fn)

34598374
267902775
241829427
9089471


In [ ]:
total_tn

267902775

In [ ]:

print(precision)

0.0530972554994664


In [ ]:

print(recall)

0.9825724787846257


In [ ]:
# def is_overlapping(seg1,seg2):
#   if (seg1[1] < seg2[0] or seg2[1] < seg1[0]):
#     return 0
#   else:
#     return 1

# def count_overlap(seg1,seg2):
#   first = seg1
#   second = seg2
#   if (seg1[0] > seg2[0]):
#     first = seg2
#     second = seg1
#   return  first[1] - second[0] + 1

# def count_total_overlap(seg,peaks):
#   total = 0
#   for i in range(len(peaks)):
#     if (peaks[i][0] > seg[1]):
#       break
#     else:
#       if (is_overlapping(seg,peaks[i])):
#         total += count_overlap(seg,peaks[i])
#   return total
  

precision =  0.0530972554994664
recall =  0.9825724787846257


In [ ]:
# FN = 0
# FP = 0
# TP = 0
# TN = 0
# for i in range(max_positions):
#   if (labels[i] == -1):
#     continue
#   else:
#     if (labels[i] == 1 and preds[i] == 0):
#       FN += 1
#     elif (labels[i] == 1 and preds[i] == 1):
#       TP += 1
#     elif (labels[i] == 0 and preds[i] == 0):
#       TN += 1
#     elif (labels[i] == 0 and preds[i] == 1):
#       FP += 1


In [ ]:
(lab == -1).sum()

0